In [3]:
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

pd.options.display.max_rows = 4000
pd.options.display.max_seq_items = 2000

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [4]:
path_contracts_lookup = '/Users/marcelmuller/Documents/Uni/Master/Semester_9_SS_18/Masterarbeit/contractsWithERCFlags.csv'
path_block_times = '/Users/marcelmuller/Documents/Uni/Master/Semester_9_SS_18/Masterarbeit/blockTimes.csv'

## sta->CtX- Why is this happening?
We found lots of example of this occurring. Let's find out why this happens

In [5]:
transitions_543 = pd.read_csv('/Users/marcelmuller/Documents/Uni/Master/Semester_9_SS_18/Masterarbeit/parity_transactions/mr_2018-04-13 00:00:00_in_transactions5430000-5439999_7_transitions.csv', index_col=0)

/Users/marcelmuller/Documents/Uni/Master/Semester_9_SS_18/Masterarbeit/distributed_ledger_process_miner/dl-pm/lib/python3.6/site-packages/numpy/lib/arraysetops.py:518: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [7]:
staC = transitions_543[transitions_543['transition']=='sta->CtC']


In [8]:
staC.head()

,total_pos,timestamp,transition
268694,5430554000122985,1523584798,sta->CtC
1554972,5433101000769178,1523621913,sta->CtC
2422988,5434778001205238,1523646714,sta->CtC
2428160,5434788001207835,1523646899,sta->CtC
2662742,5435275001325714,1523654355,sta->CtC


In [10]:
block_times = pd.read_csv(path_block_times, index_col=0)
transaction_lookup = pd.read_csv('/Users/marcelmuller/Documents/Uni/Master/Semester_9_SS_18/Masterarbeit/transaction_lookup.csv')

/Users/marcelmuller/Documents/Uni/Master/Semester_9_SS_18/Masterarbeit/distributed_ledger_process_miner/dl-pm/lib/python3.6/site-packages/numpy/lib/arraysetops.py:518: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [11]:
log_543 = pd.read_csv('/Users/marcelmuller/Documents/Uni/Master/Semester_9_SS_18/Masterarbeit/parity_transactions/ps_transactions5430000-5439999_event_log.csv', index_col=0)

/Users/marcelmuller/Documents/Uni/Master/Semester_9_SS_18/Masterarbeit/distributed_ledger_process_miner/dl-pm/lib/python3.6/site-packages/numpy/lib/arraysetops.py:518: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [12]:
candidate_tx_hashes_staC = log_543.merge(staC, on='total_pos').merge(transaction_lookup, left_on='transaction_id', right_on='id')['transaction_hash']
candidate_tx_hash_staC_list = candidate_tx_hashes_staC.values.tolist()

In [13]:
raw_543 = pd.read_csv('/Users/marcelmuller/Documents/Uni/Master/Semester_9_SS_18/Masterarbeit/parity_transactions/transactions5430000-5439999.csv')

/Users/marcelmuller/Documents/Uni/Master/Semester_9_SS_18/Masterarbeit/distributed_ledger_process_miner/dl-pm/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0,1,7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [14]:
contracts_lookup = pd.read_csv(path_contracts_lookup, index_col=0)

/Users/marcelmuller/Documents/Uni/Master/Semester_9_SS_18/Masterarbeit/distributed_ledger_process_miner/dl-pm/lib/python3.6/site-packages/numpy/lib/arraysetops.py:518: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [16]:
res_543 = raw_543[raw_543['transactionHash'].isin(candidate_tx_hash_staC_list)].merge(contracts_lookup, left_on='action.from', right_on='result.address', how='left').merge(contracts_lookup, how='left', left_on='action.to', right_on='result.address')
res_543.head(10)

,action.address,action.balance,action.callType,action.from,action.gas,action.init,action.input,action.refundAddress,action.to,action.value,blockHash,blockNumber,error,result,result.address_x,result.code,result.gasUsed,result.output,subtraces,traceAddress,transactionHash,transactionPosition,type,result.address_y,isERC20_x,timestamp_x,result.address,isERC20_y,timestamp_y
0,NaN,NaN,NaN,0x1726b43acad4cddbad3a8877a5eb40693564646b,0x29aabf,0x606060405260056001556000600a819055600b819055...,NaN,NaN,NaN,0x0,0x76be8658feb97393e6114a978001454403e929eb0911...,5430554,NaN,NaN,0xc76119f30ad3423075a9cfe2a79ed3cffb031d07,0x606060405236156102a35763ffffffff60e060020a60...,0x285eb7,NaN,1,[],0xd81dd0a07ac8ee8d8962ce7c44358dbd839183212efc...,74.0,create,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,call,0xc76119f30ad3423075a9cfe2a79ed3cffb031d07,0x26d6d2,NaN,0x04bbc255,NaN,0x6781ae4fb9698727f4fd1a10144c356ad4d40a45,0x0,0x76be8658feb97393e6114a978001454403e929eb0911...,5430554,NaN,NaN,NaN,NaN,0xb4,0x00000000000000000000000000000000000000000000...,0,[0],0xd81dd0a07ac8ee8d8962ce7c44358dbd839183212efc...,74.0,call,0xc76119f30ad3423075a9cfe2a79ed3cffb031d07,False,1.523585e+09,0x6781ae4fb9698727f4fd1a10144c356ad4d40a45,False,1.523585e+09
2,NaN,NaN,NaN,0x8af0f8b1511da6824241ce8a2b1c32034d53bd5b,0x3c2360,0x606060405234156200001057600080fd5b6040516080...,NaN,NaN,NaN,0x0,0x1405f53cc14d7ba9936b58deb3ed7fdf6c10774a7121...,5433101,NaN,NaN,0xfb840fe0857587910cf0ec1bdbdb4f356f3cd566,0x6060604052600436106103975763ffffffff60e06002...,0x2f4ef7,NaN,1,[],0xb712252df23ca1d09d2f0d5255f4e9e2ad0534239667...,43.0,create,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,call,0xfb840fe0857587910cf0ec1bdbdb4f356f3cd566,0x394625,NaN,0x313ce567,NaN,0x8b2da472ddcd2aa76e70b86d026e65b176902941,0x0,0x1405f53cc14d7ba9936b58deb3ed7fdf6c10774a7121...,5433101,NaN,NaN,NaN,NaN,0x222,0x00000000000000000000000000000000000000000000...,0,[0],0xb712252df23ca1d09d2f0d5255f4e9e2ad0534239667...,43.0,call,0xfb840fe0857587910cf0ec1bdbdb4f356f3cd566,False,1.523622e+09,0x8b2da472ddcd2aa76e70b86d026e65b176902941,True,1.523622e+09
4,NaN,NaN,NaN,0x966917a4c19888a012786d3220126090b8236da2,0x14826d,0x606060405262989680600455341561001657600080fd...,NaN,NaN,NaN,0x0,0xf72fe4fe78f02e71b40730a6ae6a01cf100b8f863171...,5434778,NaN,NaN,0x7f0910fd07961efd87cdaf7f99a51c174dc8684d,0x6060604052600436106102005763ffffffff60e06002...,0x14826d,NaN,1,[],0x244347c7a97452230e676ae2c4d8bfa5708b7918b1df...,115.0,create,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,call,0x7f0910fd07961efd87cdaf7f99a51c174dc8684d,0x12efd9,NaN,0xd5abeb01,NaN,0xffe1fb86ad7c08853e8de622d161f6bdaaf695f8,0x0,0xf72fe4fe78f02e71b40730a6ae6a01cf100b8f863171...,5434778,NaN,NaN,NaN,NaN,0x45e,0x0000000000000000000000000000000000000000033b...,0,[0],0x244347c7a97452230e676ae2c4d8bfa5708b7918b1df...,115.0,call,0x7f0910fd07961efd87cdaf7f99a51c174dc8684d,False,1.523647e+09,0xffe1fb86ad7c08853e8de622d161f6bdaaf695f8,True,1.523647e+09
6,NaN,NaN,NaN,0x966917a4c19888a012786d3220126090b8236da2,0x288aa1,0x60606040526298968060045534156200001757600080...,NaN,NaN,NaN,0x0,0x73dc013c3d5308c694386dc76e15a12530f6db44846f...,5434788,NaN,NaN,0x2f7cb09b775dbdc80b6b8dc6e8512dbf8e3ff30c,0x6060604052600436106102585763ffffffff60e06002...,0x288aa1,NaN,1,[],0x368cadf1eee8fe4271a666939ccb79e03f80d25c5b3a...,132.0,create,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,call,0x2f7cb09b775dbdc80b6b8dc6e8512dbf8e3ff30c,0x26a7d5,NaN,0xd5abeb01,NaN,0xffe1fb86ad7c08853e8de622d161f6bdaaf695f8,0x0,0x73dc013c3d5308c694386dc76e15a12530f6db44846f...,5434788,NaN,NaN,NaN,NaN,0x45e,0x0000000000000000000000000000000000000000033b...,0,[0],0x368cadf1eee8fe4271a666939ccb79e03f80d25c5b3a...,132.0,call,0x2f7cb09b775dbdc80b6b8dc6e8512dbf8e3ff30c,False,1.523647e+09,0xffe1fb86ad7c08853e8de622d161f6bdaaf695f8,True,1.523647e+09
8,NaN,NaN,NaN,0x6ec4e85bcaed37a4093a0bb82d4beca1d74a6a7c,0x24b59a,0x606060405260058054600160a060020a03191673c3ed...,NaN,NaN,NaN,0x0,0xe1cb61f45d2769dbf2b6fd5fc8f2e4fdee3c28039daa...,5435275,NaN,NaN,0x775ff1e18dbeb4fd881c3f4049e675016786

In [18]:
a = res_543[['action.from', 'action.to', 'result.address_x', 'isERC20_x', 'result.address_y', 'isERC20_x', 'subtraces', 'type', 'result.address', 'transactionHash']]
a

,action.from,action.to,result.address_x,isERC20_x,result.address_y,isERC20_x,subtraces,type,result.address,transactionHash
0,0x1726b43acad4cddbad3a8877a5eb40693564646b,NaN,0xc76119f30ad3423075a9cfe2a79ed3cffb031d07,NaN,NaN,NaN,1,create,NaN,0xd81dd0a07ac8ee8d8962ce7c44358dbd839183212efc...
1,0xc76119f30ad3423075a9cfe2a79ed3cffb031d07,0x6781ae4fb9698727f4fd1a10144c356ad4d40a45,NaN,False,0xc76119f30ad3423075a9cfe2a79ed3cffb031d07,False,0,call,0x6781ae4fb9698727f4fd1a10144c356ad4d40a45,0xd81dd0a07ac8ee8d8962ce7c44358dbd839183212efc...
2,0x8af0f8b1511da6824241ce8a2b1c32034d53bd5b,NaN,0xfb840fe0857587910cf0ec1bdbdb4f356f3cd566,NaN,NaN,NaN,1,create,NaN,0xb712252df23ca1d09d2f0d5255f4e9e2ad0534239667...
3,0xfb840fe0857587910cf0ec1bdbdb4f356f3cd566,0x8b2da472ddcd2aa76e70b86d026e65b176902941,NaN,False,0xfb840fe0857587910cf0ec1bdbdb4f356f3cd566,False,0,call,0x8b2da472ddcd2aa76e70b86d026e65b176902941,0xb712252df23ca1d09d2f0d5255f4e9e2ad0534239667...
4,0x966917a4c19888a012786d3220126090b8236da2,NaN,0x7f0910fd07961efd87cdaf7f99a51c174dc8684d,NaN,NaN,NaN,1,create,NaN,0x244347c7a97452230e676ae2c4d8bfa5708b7918b1df...
5,0x7f0910fd07961efd87cdaf7f99a51c174dc8684d,0xffe1fb86ad7c08853e8de622d161f6bdaaf695f8,NaN,False,0x7f0910fd07961efd87cdaf7f99a51c174dc8684d,False,0,call,0xffe1fb86ad7c08853e8de622d161f6bdaaf695f8,0x244347c7a97452230e676ae2c4d8bfa5708b7918b1df...
6,0x966917a4c19888a012786d3220126090b8236da2,NaN,0x2f7cb09b775dbdc80b6b8dc6e8512dbf8e3ff30c,NaN,NaN,NaN,1,create,NaN,0x368cadf1eee8fe4271a666939ccb79e03f80d25c5b3a...
7,0x2f7cb09b775dbdc80b6b8dc6e8512dbf8e3ff30c,0xffe1fb86ad7c08853e8de622d161f6bdaaf695f8,NaN,False,0x2f7cb09b775dbdc80b6b8dc6e8512dbf8e3ff30c,False,0,call,0xffe1fb86ad7c08853e8de622d161f6bdaaf695f8,0x368cadf1eee8fe4271a666939ccb79e03f80d25c5b3a...
8,0x6ec4e85bcaed37a4093a0bb82d4beca1d74a6a7c,NaN,0x775ff1e18dbeb4fd881c3f4049e675016786c242,NaN,NaN,NaN,8,create,NaN,0x7890c4f7823c09419034e94849e41a96c9b8ea830779...
9,0x775ff1e18dbeb4fd881c3f4049e675016786c242,0x1d3b2638a7cc9f2cb3d298a3da7a90b67e5506ed,NaN,False,0x775ff1e18dbeb4fd881c3f4049e675016786c242,False,0,call,0x1d3b2638a7cc9f2cb3d298a3da7a90b67e5506ed,0x7890c4f7823c09419034e94849e41a96c9b8ea830779...


In [26]:
pd.options.display.max_colwidth=10000

In [27]:
a.iloc[[8]]['transactionHash']

8    0x7890c4f7823c09419034e94849e41a96c9b8ea830779f2febe6c7cb4eb38fb01
Name: transactionHash, dtype: object

In [28]:
a[a['transactionHash']=='0x7890c4f7823c09419034e94849e41a96c9b8ea830779f2febe6c7cb4eb38fb01']

,action.from,action.to,result.address_x,isERC20_x,result.address_y,isERC20_x,subtraces,type,result.address,transactionHash
8,0x6ec4e85bcaed37a4093a0bb82d4beca1d74a6a7c,NaN,0x775ff1e18dbeb4fd881c3f4049e675016786c242,NaN,NaN,NaN,8,create,NaN,0x7890c4f7823c09419034e94849e41a96c9b8ea830779f2febe6c7cb4eb38fb01
9,0x775ff1e18dbeb4fd881c3f4049e675016786c242,0x1d3b2638a7cc9f2cb3d298a3da7a90b67e5506ed,NaN,False,0x775ff1e18dbeb4fd881c3f4049e675016786c242,False,0,call,0x1d3b2638a7cc9f2cb3d298a3da7a90b67e5506ed,0x7890c4f7823c09419034e94849e41a96c9b8ea830779f2febe6c7cb4eb38fb01
10,0x775ff1e18dbeb4fd881c3f4049e675016786c242,0x1d3b2638a7cc9f2cb3d298a3da7a90b67e5506ed,NaN,False,0x775ff1e18dbeb4fd881c3f4049e675016786c242,False,0,call,0x1d3b2638a7cc9f2cb3d298a3da7a90b67e5506ed,0x7890c4f7823c09419034e94849e41a96c9b8ea830779f2febe6c7cb4eb38fb01
11,0x775ff1e18dbeb4fd881c3f4049e675016786c242,0x3dbdc81a6edc94c720b0b88fb65dbd7e395fdcf6,NaN,False,0x775ff1e18dbeb4fd881c3f4049e675016786c242,False,0,call,0x3dbdc81a6edc94c720b0b88fb65dbd7e395fdcf6,0x7890c4f7823c09419034e94849e41a96c9b8ea830779f2febe6c7cb4eb38fb01
12,0x775ff1e18dbeb4fd881c3f4049e675016786c242,0x1d3b2638a7cc9f2cb3d298a3da7a90b67e5506ed,NaN,False,0x775ff1e18dbeb4fd881c3f4049e675016786c242,False,0,call,0x1d3b2638a7cc9f2cb3d298a3da7a90b67e5506ed,0x7890c4f7823c09419034e94849e41a96c9b8ea830779f2febe6c7cb4eb38fb01
13,0x775ff1e18dbeb4fd881c3f4049e675016786c242,0x3dbdc81a6edc94c720b0b88fb65dbd7e395fdcf6,NaN,False,0x775ff1e18dbeb4fd881c3f4049e675016786c242,False,0,call,0x3dbdc81a6edc94c720b0b88fb65dbd7e395fdcf6,0x7890c4f7823c09419034e94849e41a96c9b8ea830779f2febe6c7cb4eb38fb01
14,0x775ff1e18dbeb4fd881c3f4049e675016786c242,0x1d3b2638a7cc9f2cb3d298a3da7a90b67e5506ed,NaN,False,0x775ff1e18dbeb4fd881c3f4049e675016786c242,False,0,call,0x1d3b2638a7cc9f2cb3d298a3da7a90b67e5506ed,0x7890c4f7823c09419034e94849e41a96c9b8ea830779f2febe6c7cb4eb38fb01
15,0x775ff1e18dbeb4fd881c3f4049e675016786c242,0x3dbdc81a6edc94c720b0b88fb65dbd7e395fdcf6,NaN,False,0x775ff1e18dbeb4fd881c3f4049e675016786c242,False,0,call,0x3dbdc81a6edc94c720b0b88fb65dbd7e395fdcf6,0x7890c4f7823c09419034e94849e41a96c9b8ea830779f2febe6c7cb4eb38fb01
16,0x775ff1e18dbeb4fd881c3f4049e675016786c242,0x3dbdc81a6edc94c720b0b88fb65dbd7e395fdcf6,NaN,False,0x775ff1e18dbeb4fd881c3f4049e675016786c242,False,0,call,0x3dbdc81a6edc94c720b0b88fb65dbd7e395fdcf6,0x7890c4f7823c09419034e94849e41a96c9b8ea830779f2febe6c7cb4eb38fb01


Looking into it with Etherscan we can observe a contract creating, which gets filtered out by us since we are not using anything else than call types, followed by a next event from that contract to another contract. Computation is correct, but how can contracts start themselves?